In [1]:
import robosandbox as rsb
import numpy as np
import swift
import os


## Generic Models

In [ ]:
qlim_l = np.array([-np.pi, -np.pi, -np.pi, -np.pi])
qlim_u = np.array([np.pi, np.pi, np.pi, np.pi])
qlim = np.vstack((qlim_l, qlim_u))
G4 = rsb.models.URDF.GenericDH(
    dofs=4,
    a=[0, -0.4, -0.4, -0.4],
    d=[0.4, 0, 0, 0],
    alpha=[np.pi / 2, 0, 0, 0],
    qlim=qlim,
)
G4.q = np.array([np.pi / 2, -np.pi / 2, 0, 0])


axis is [-0.  0.  0.]
angle is 0.0
axis is [ 0. -1.  0.]
angle is 1.5707963267948966
axis is [ 0. -1.  0.]
angle is 1.5707963267948966
axis is [ 0. -1.  0.]
angle is 1.5707963267948966


/var/folders/x1/r8g1dmlj1dn7lz6z4bdx_mfr0000gn/T/ipykernel_10391/1949167462.py:4: UserWarning:

Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.



## Commercial models

### Panda

In [38]:
panda = rsb.models.URDF.Panda()
panda.q = panda.qz
env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(panda)

0

### Puma560

In [36]:
puma = rsb.models.URDF.Puma560()
puma.q = puma.qr
env = swift.Swift()
env.launch(realtime=True, browser="notebook")
env.add(puma)

0

## Read from URDF File

In [29]:
# get the current working directory
pwd = os.getcwd()
tld = os.path.join(pwd, "data", "SO101")

SO101 = rsb.models.URDF.Model(
    name="SO101",
    file_path="so101_new_calib.urdf",
    tld=tld,
)
SO101.q = np.zeros(6)
print(SO101)
SO101.plot(q=SO101.q)

ERobot: so101_new_calib, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬───────────┬───────┬───────────┬───────────────────────────────────────────────────────────────────────┐
│ link │   link    │ joint │  parent   │                          ETS: parent to link                          │
├──────┼───────────┼───────┼───────────┼───────────────────────────────────────────────────────────────────────┤
│    0 │ base      │       │ BASE      │ SE3()                                                                 │
│    1 │ shoulder  │     0 │ base      │ SE3(0.02079, -0.02307, 0.09488; -180°, 3.459e-14°, 90°) ⊕ Rz(q0)      │
│    2 │ upper_arm │     1 │ shoulder  │ SE3(-0.0304, -0.01828, -0.0542; 90°, -90°, 180°) ⊕ Rz(q1)             │
│    3 │ lower_arm │     2 │ upper_arm │ SE3(-0.1126, -0.028, 2.463e-16; -7.037e-14°, 3.3e-14°, 90°) ⊕ Rz(q2)  │
│    4 │ wrist     │     3 │ lower_arm │ SE3(-0.1349, 0.0052, 1.652e-16; 1.861e-13°, 1.64e-13°, -90°) ⊕ Rz(q3) │
│    5 │ gripper   │  

Swift backend, t = 0.05, scene:
  so101_new_calib

In [4]:
print(puma._T)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [70]:
env = swift.Swift()
env.launch(realtime=True, browser="notebook")

SO101._T = np.array(
    [
        [1, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
puma._T = np.array(
    [
        [1, 0, 0, 0.5],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
panda._T = np.array(
    [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
G4._T = np.array(
    [
        [1, 0, 0, 1.5],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ]
)
env.add(G4)
env.add(panda)
env.add(SO101)
env.add(puma)
# env.hold()
env.set_camera_pose([-0.5, -2, 0.5], [1, 0.0, 0.5])

In [84]:
import robosandbox.performance.workspace.WorkSpace as ws

workspace = ws(SO101)
global_mu = workspace.global_indice(method="invcondition")
print("Global Indices (invcondition):", global_mu)
workspace.plot(color="invcondition")
print(SO101.qlim)

Global Indices (invcondition): 0.017309849907618968


[[-1.91986  -1.74533  -1.74533  -1.65806  -2.79253  -0.174533]
 [ 1.91986   1.74533   1.5708    1.65806   2.79253   1.74533 ]]
